In [ ]:
import nbformat, re
#from nbconvert.preprocessors import ExecutePreprocessor, Preprocessor
from nbconvert import MarkdownExporter
from traitlets.config import Config
from pathlib import Path

In [ ]:
def _exporter():
    exporter = MarkdownExporter(Config())
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = 'jekyll.tpl'
    exporter.template_path.append(str(Path.cwd().absolute()))
    return exporter

In [ ]:
_re_title = re.compile(r'^\s*#\s+([^\n]*\n)\s*')

def process_title(cell):
    if cell['cell_type'] == 'code': return cell
    if _re_title.search(cell['source']):
        cell['source'] = '---\n' + _re_title.sub(r'- title: \1', cell['source']) + '\n---'
    return cell

In [ ]:
def apply_all(x, fs, **kwargs):
    for f in fs: x = f(x, **kwargs)
    return x

In [ ]:
def convert(fname, dest=None, cell_procs=None):
    fname = Path(fname)
    if dest is None: dest = fname.with_suffix('.md')
    if cell_procs is None: cell_procs = [process_title]
    with open(fname,'r') as f: nb = nbformat.reads(f.read(), as_version=4)
    nb['cells'] = [apply_all(c, cell_procs) for c in nb['cells']]
    with open(dest,'w') as f: f.write(_exporter().from_notebook_node(nb)[0])

In [ ]:
convert(Path.cwd()/'delegation.ipynb')